In [1]:
from pyrouge import Rouge155
from pprint import pprint

ref_texts = {'A': "Poor nations pressurise developed countries into granting trade subsidies.",
             'B': "Developed countries should be pressurized. Business exemptions to poor nations.",
             'C': "World's poor decide to urge developed nations for business concessions."}
summary_text = "Poor nations demand trade subsidies from developed nations."

In [2]:
r = Rouge155()

In [3]:
base = '/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/'
r.system_dir = base + 'system_multi'
r.model_dir = base + 'model_multi'
r.system_filename_pattern = 'SetSummary.(\d+).txt'
r.model_filename_pattern = 'SetSummary.#ID#.[A-Z].txt'

In [ ]:
output = r.convert_and_evaluate()

In [ ]:
print output

In [ ]:
output_dict = r.output_to_dict(output)

In [ ]:
print output_dict

In [ ]:
# Let's try to rename the model summaries so that they match what we know
import os
base = '/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/'

In [ ]:
# Only run once, won't work again
modelBase = base + 'model'
for name in os.listdir(modelBase):
    tmp = name.split('.')
    newName = 'Summary.{}.{}'.format(tmp[0][1:], tmp[-1])
    newFile = os.path.join(modelBase, newName)
    print name
    oldFile = os.path.join(modelBase, name)
    os.rename(oldFile, newFile)

In [ ]:
for name in os.listdir(modelBase):
    newName = 'Set' + name + '.txt'
    newFile = os.path.join(modelBase, newName)
    oldFile = os.path.join(modelBase, name)
    os.rename(oldFile, newFile)

In [ ]:
from collections import Counter

In [ ]:
# Only run once, won't work again
modelBase = base + 'model_single'
setCount = Counter() # Keeps count of documents in a set
docIDCount = Counter()
for name in os.listdir(modelBase):
    tmp = name.split('.')
    ID = tmp[0][1:]
    Person = tmp[4]
    DOCID = tmp[-2][3:] + tmp[-1]
    newName = 'Summary.{}{}.{}.txt'.format(ID,DOCID , Person)
    setCount[ID] += 1
    docIDCount[DOCID] += 1
    newFile = os.path.join(modelBase, newName)
    print name
    oldFile = os.path.join(modelBase, name)
    os.rename(oldFile, newFile)

In [ ]:
# Take the original documents and rename their containing forlder
# Only run once, won't work again
modelBase = base + 'docs'
_, dirs, _ = os.walk(modelBase).next()
for dir in dirs:
    ID = dir[1:-1]
    newFile = os.path.join(modelBase, ID)
    print ID
    oldFile = os.path.join(modelBase, dir)
    os.rename(oldFile, newFile)

In [ ]:
modelBase = base + 'docs'
_, dirs, _ = os.walk(modelBase).next()
for dir in dirs:
    for name in os.listdir(os.path.join(modelBase, dir)):
        tmp = name.split('.')
        ID = tmp[0][3:] + tmp[1] + '.txt'
        newFile = os.path.join(modelBase, dir, ID)
        print newFile
        oldFile = os.path.join(modelBase, dir, name)
        os.rename(oldFile, newFile)

In [ ]:
# Now we want to take the documents and extract the data and save as a 
# text file with the right document ids!
from bs4 import BeautifulSoup as BSHTML

modelBase = base + 'docs'
_, dirs, _ = os.walk(modelBase).next()
for dir in dirs:
    for name in os.listdir(os.path.join(modelBase, dir)):
        # We have access to each file, so let's read it and rewrite it
        file = os.path.join(modelBase, dir, name)
        with open(file, 'r') as txt:
            HTML = BSHTML(txt.read(), 'xml')
            text = HTML.TEXT.text
            sentences = [s.replace('\n', '') for s in text.split('. ')]
            for s in sentences:
                print s
            with open(os.path.join(modelBase, dir, 'parsed.' + name), 'w') as f:
                for s in sentences:
                    f.write("{}\n".format(s))

In [58]:
def createSummaries(sum_algo, abs_path, out_path, k = 5, multiDocument=False):
    # Extracted from the input folder name 
    setID = abs_path.split('/')[-1]
    
    # Stores DocumentIDs
    docIDs = []
    
    # Create document set that we wish to evaluate
    D = []
    for filename in os.listdir(abs_path):
        # Only take files that we've parsed before!
        tmp = filename.split('.')
        if tmp[0] == 'parsed':
            docIDs.append(tmp[1])
            filepath = os.path.join(abs_path, filename)
            with open(filepath) as inputDoc:
                D.append(inputDoc.readlines())
            
    # Pass this to the algorithm which should return the summary as
    # a list of sentences.
    if multiDocument:
        summary = sum_algo(D, k)
        # Write out the summary
        filepath = os.path.join(out_path, "SetSummary.{}.txt".format(setID))
        with open(filepath, 'w+') as out:
            for s in summary:
                out.write("{}.\n".format(s.strip()))
    else:
        for i in range(len(D)):
            summary = sum_algo([D[i]], k)
            filepath = os.path.join(out_path, "Summary.{}.txt".format(docIDs[i]))
            with open(filepath, 'w+') as out:
                for s in summary:
                    out.write("{}.\n".format(s.strip()))

In [59]:
import sys 
project_package = '/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/'
if project_package not in sys.path:
    sys.path.append(project_package)
from summarizer import utils
from summarizer import grasshopper

In [60]:
reload(utils)
reload(grasshopper)

<module 'summarizer.grasshopper' from '/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/summarizer/grasshopper.pyc'>

In [61]:
def grassHopper(D, k):
    # Concatenate all the documents
    D = [s for d in D for s in d]
    
    # Clean the document
    cleanDoc, mapping = utils.cleanDocument(D)
    WClean = grasshopper.docToMatrix(cleanDoc, sim_fun=utils.threshHoldCosineSim)
    
    # According to the paper, alpha = 0.25 and lambda = 0.5 where the 
    # best parameters
    lamb = 0.5
    alpha = 0.25
    r = utils.decayDistribution(alpha, len(WClean))
    results = grasshopper.grasshopper(WClean, r, lamb, 5)
    
    # Extract the summary
    summary = []
    for (i, p) in sorted(results, key=lambda x: x[0]):
        summary.append(D[i])
        
    return summary

In [74]:
base = '/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/'
outpath = base + 'grass_out'
inbase = base + 'docs/'
for folder in os.listdir(inbase):
    inpath = inbase + folder
    try:
        createSummaries(grassHopper, inpath, outpath, multiDocument=True)
    except:
        print "Failed with {}".format(inpath)

Failed with /home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/docs/30045
Failed with /home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/docs/31033


In [70]:
r = Rouge155()

base = '/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/'
r.system_dir = base + 'grass_out'
r.model_dir = base + 'model_multi'
r.system_filename_pattern = 'SetSummary.(\d+).txt'
r.model_filename_pattern = 'SetSummary.#ID#.[A-Z].txt'

output = r.convert_and_evaluate()

2015-12-06 18:36:10,334 [MainThread  ] [INFO ]  Writing summaries.
INFO:global:Writing summaries.
2015-12-06 18:36:10,337 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmpR_MuQw/system and model files to /tmp/tmpR_MuQw/model.
INFO:global:Processing summaries. Saving system files to /tmp/tmpR_MuQw/system and model files to /tmp/tmpR_MuQw/model.
2015-12-06 18:36:10,339 [MainThread  ] [INFO ]  Processing files in /home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/grass_out.
INFO:global:Processing files in /home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/grass_out.
2015-12-06 18:36:10,340 [MainThread  ] [INFO ]  Processing SetSummary.30001.txt.
INFO:global:Processing SetSummary.30001.txt.
2015-12-06 18:36:10,345 [MainThread  ] [INFO ]  Processing SetSummary.30002.txt.
INFO:global:Processing SetSummary.30002.txt.
2015-12-06 18:36:10,347 [MainThread  ] [INFO ]  Saved pr

In [75]:
print output

---------------------------------------------
1 ROUGE-1 Average_R: 0.60203 (95%-conf.int. 0.54653 - 0.65753)
1 ROUGE-1 Average_P: 0.56967 (95%-conf.int. 0.56039 - 0.57895)
1 ROUGE-1 Average_F: 0.58456 (95%-conf.int. 0.55337 - 0.61574)
---------------------------------------------
1 ROUGE-2 Average_R: 0.39196 (95%-conf.int. 0.36213 - 0.42178)
1 ROUGE-2 Average_P: 0.37125 (95%-conf.int. 0.37115 - 0.37136)
1 ROUGE-2 Average_F: 0.38077 (95%-conf.int. 0.36669 - 0.39485)
---------------------------------------------
1 ROUGE-3 Average_R: 0.32683 (95%-conf.int. 0.31153 - 0.34214)
1 ROUGE-3 Average_P: 0.31019 (95%-conf.int. 0.30088 - 0.31951)
1 ROUGE-3 Average_F: 0.31783 (95%-conf.int. 0.31547 - 0.32019)
---------------------------------------------
1 ROUGE-4 Average_R: 0.28647 (95%-conf.int. 0.27838 - 0.29456)
1 ROUGE-4 Average_P: 0.27221 (95%-conf.int. 0.25889 - 0.28554)
1 ROUGE-4 Average_F: 0.27874 (95%-conf.int. 0.27558 - 0.28191)
---------------------------------------------
1 ROUGE-L Aver